In [1]:
import json
import pandas as pd
import xlsxwriter
import numpy as np
import time
import os
import sys
from statistics import mean

In [2]:
rootPath = "D:/Cloud/Onedrive/Pessoal/Acadêmico/FEUP/2019-2020-PDISS_DISS/"
videoPath = "_research/Results/20200218/ImageJ/561_H2O_100ul/"
fileName = "561_D2_H2O_100ul_Series001_RAW_ch00.txt"
dataFile = f"{rootPath}{videoPath}{fileName}"

In [3]:
def import_file(filename):

    f = open(filename)
    f_list = list(f)
    first_trajectory_row = f_list.index('%% Trajectory 1\n')
    f.close()

    rows_to_skip = first_trajectory_row

    data = pd.read_csv(
        filename,
        skiprows=rows_to_skip,
        delim_whitespace=True,
        usecols=[0,1, 2],
        names=["frame", "x", "y"],
#         decimal=".",
    )
    return data

data = import_file(dataFile)
data.head()

,frame,x,y
0,%%,Trajectory,1
1,%%,frame,x
2,0,"445,255188","1,994863"
3,1,"445,022858","4,404033"
4,2,"446,188019","5,049350"


In [4]:
def clean_data(data):
    data.loc[:, "x"].replace(
        to_replace="Trajectory", value=0, regex=True, inplace=True
    )

    # Delete rows that have the value 'frame'
    indexNames = data[data['x'] == 'frame'].index
    data.drop(indexNames, inplace=True)

    return data

data = clean_data(data)
data.head()

,frame,x,y
0,%%,0,1
2,0,"445,255188","1,994863"
3,1,"445,022858","4,404033"
4,2,"446,188019","5,049350"
5,3,"445,844421","7,074995"


In [5]:
def correct_separator(data):
    # data.loc[:, "x"].replace(to_replace=",", value=".",
    #                          regex=True, inplace=True)
    data.replace(to_replace=",", value=".",
                 regex=True, inplace=True)
    return data

data = correct_separator(data)
data.head()

,frame,x,y
0,%%,0,1
2,0,445.255188,1.994863
3,1,445.022858,4.404033
4,2,446.188019,5.049350
5,3,445.844421,7.074995


In [6]:
def remove_index(data):
    data.reset_index(drop=True, inplace=True)
    return data

data = remove_index(data)
data.head()

,frame,x,y
0,%%,0,1
1,0,445.255188,1.994863
2,1,445.022858,4.404033
3,2,446.188019,5.049350
4,3,445.844421,7.074995


In [7]:
def split_trajectories(data):
    result = []
    last_index = 0
    for ind, row in data.iterrows():
        if data.loc[ind, "x"] == 0 and ind > 0:
            index = ind
            result.append(data.iloc[last_index:index])
            last_index = ind

    result.append(data.iloc[last_index:index])

    return result

data_list = split_trajectories(data)
len(data_list)

2911

In [8]:
data_list[0]

,frame,x,y
0,%%,0,1
1,0,445.255188,1.994863
2,1,445.022858,4.404033
3,2,446.188019,5.049350
4,3,445.844421,7.074995
5,4,443.597351,5.036721
6,5,442.252075,5.532090
7,6,443.687164,3.512489
8,7,443.604828,5.429587
9,8,443.699524,3.412457


In [12]:
def add_trajectory_nr(data):
    if len(data)>0:
        data.reset_index(drop=True, inplace=True)
        data.insert(loc=0, column='particle', value=data.at[0,'y'])
        # data['particle'] = data.at[0,'y']
        data.drop([0],inplace=True)
    
    return data

for data in data_list:
    add_trajectory_nr(data)
    # data.drop([0],inplace=True)

data_list[0]

,particle,frame,x,y
1,1,0,445.255188,1.994863
2,1,1,445.022858,4.404033
3,1,2,446.188019,5.049350
4,1,3,445.844421,7.074995
5,1,4,443.597351,5.036721
6,1,5,442.252075,5.532090
7,1,6,443.687164,3.512489
8,1,7,443.604828,5.429587
9,1,8,443.699524,3.412457
10,1,9,441.822144,4.909852


In [13]:
def get_valid_trajectories(data):
    result = []
    for trajectory in data:
        if len(trajectory) >= 590:
            result.append(trajectory)

    return result

data_list = get_valid_trajectories(data_list)
len(data_list)

8

In [25]:
def to_numeric(data):
    data["particle"] = pd.to_numeric(data["particle"])
    data["frame"] = pd.to_numeric(data["frame"])
    data["x"] = pd.to_numeric(data["x"])
    data["y"] = pd.to_numeric(data["y"])
    return data

for data in data_list:
    data = to_numeric(data)

data_list[0].head()

,particle,frame,x,y
1,14,0,267.523956,36.206261
2,14,1,266.769745,38.172245
3,14,2,268.312408,36.173717
4,14,3,267.643372,38.647327
5,14,4,268.294067,37.469563


In [31]:
full_data = pd.concat(data_list)
full_data

,particle,frame,x,y
1,14,0,267.523956,36.206261
2,14,1,266.769745,38.172245
3,14,2,268.312408,36.173717
4,14,3,267.643372,38.647327
5,14,4,268.294067,37.469563
...,...,...,...,...
595,146,601,212.085663,477.182343
596,146,602,213.531937,476.796753
597,146,603,212.162201,476.769989
598,146,604,211.293259,476.177094


In [33]:
import trackpy as tp
mpp = 160/512    # Data from video
fps = 30         # Data from video
mlt = fps * 10   # Analysis time (in seconds)
msd = tp.imsd(pd.concat(data_list), mpp, 30, mlt)
msd.head()

,14,26,56,62,99,107,141,146
lag time [s],,,,,,,,
0.033333,0.433101,0.443323,0.374815,0.447934,0.386011,0.377463,0.356432,0.348206
0.066667,0.555656,0.594187,0.402178,0.604556,0.443716,0.447019,0.352935,0.329508
0.100000,0.985808,1.090046,0.722080,1.050796,0.843761,0.806102,0.710805,0.641217
0.133333,1.119699,1.306952,0.747567,1.216781,0.927994,0.890923,0.733628,0.632308
0.166667,1.528644,1.809337,1.076573,1.643715,1.324223,1.230515,1.065676,0.934905
